In [1]:
import sqlite3
import webbrowser
conn = sqlite3.connect('database.db')

c = conn.cursor()

sql = """
DROP TABLE IF EXISTS users;
CREATE TABLE users (
           id integer unique primary key autoincrement,
           name text
);
"""
c.executescript(sql)

conn.commit()

conn.close()


In [2]:
import cv2
import numpy as np 
import sqlite3
import os

conn = sqlite3.connect('database.db')
if not os.path.exists('./Pictures/dataset'):
    os.makedirs('./Pictures/dataset')

c = conn.cursor()

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

cap = cv2.VideoCapture(0)

uname = "pawan"

c.execute('INSERT INTO users (name) VALUES (?)', (uname,))

uid = c.lastrowid

sampleNum = 0

while True:
	ret, img = cap.read()
	gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
	faces = face_cascade.detectMultiScale(gray, 1.3, 5)
	for (x,y,w,h) in faces:
		sampleNum = sampleNum+1
		cv2.imwrite("Pictures/dataset/User."+str(uid)+"."+str(sampleNum)+".jpg",gray[y:y+h,x:x+w])
		cv2.rectangle(img, (x,y), (x+w, y+h), (255,0,0), 2)
		cv2.waitKey(100)
	cv2.imshow('img',img)
	cv2.waitKey(1);
	if sampleNum > 20:
		break
cap.release()

conn.commit()

conn.close()
cv2.destroyAllWindows()

In [3]:
import os
import cv2
import numpy as np 
from PIL import Image

recognizer = cv2.face.LBPHFaceRecognizer_create()
path = 'Pictures/dataset'
if not os.path.exists('./Pictures/recognizer'):
    os.makedirs('./Pictures/recognizer')

def getImagesWithID(path):
	imagePaths = [os.path.join(path,f) for f in os.listdir(path)]
	faces = []
	IDs = []
	for imagePath in imagePaths:
		faceImg = Image.open(imagePath).convert('L')
		faceNp = np.array(faceImg,'uint8')
		ID = int(os.path.split(imagePath)[-1].split('.')[1])
		faces.append(faceNp)
		IDs.append(ID)
		cv2.imshow("training",faceNp)
		cv2.waitKey(10)
	return np.array(IDs), faces

Ids, faces = getImagesWithID(path)
recognizer.train(faces,Ids)
recognizer.save('Pictures/recognizer/trainingData.yml')
cv2.destroyAllWindows()

AttributeError: module 'cv2.cv2' has no attribute 'face'

In [ ]:
import cv2
import numpy as np 
import sqlite3
import os

conn = sqlite3.connect('database.db')
c = conn.cursor()

fname = "Pictures/recognizer/trainingData.yml"
if not os.path.isfile(fname):
    print("Please train the data first")
    exit(0)

face_cascade = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
cap = cv2.VideoCapture(0)
recognizer = cv2.face.LBPHFaceRecognizer_create()
recognizer.read(fname)

while True:
    conf = 0
    confi = 0
    ret, img = cap.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)
    for (x,y,w,h) in faces:
        cv2.rectangle(img,(x,y),(x+w,y+h),(0,255,0),3)
        ids,conf = recognizer.predict(gray[y:y+h,x:x+w])
        c.execute("select name from users where id = (?);", (ids,))
        result = c.fetchall()
        name = result[0][0]
        if conf < 35:
            confi = (100-conf)*1.27
            conf = "confidence : " + str(int(confi))
            cv2.putText(img, conf, (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (150,255,0),2)
            if(confi >= 90):
                print(name + "  You are identified")
                webbrowser.open('https://www.google.com', new=2)
                break
                cap.release()
                cv2.destroyAllWindows()
        else:
            cv2.putText(img, 'No Match', (x+2,y+h-5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255),2)
    cv2.imshow('Face Recognizer',img)
    k = cv2.waitKey(30) & 0xff
    if k == 13:
        break
        
    if confi >= 90:
        break

cap.release()
cv2.destroyAllWindows()



In [ ]:
name + "You are identified"

In [ ]:
name = "pawan"